In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Koushik\\OneDrive\\Documents\\GitHub\\mlops_project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Koushik\\OneDrive\\Documents\\GitHub\\mlops_project'

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("artifacts\data_ingestion\winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [8]:
from pathlib import Path
from dataclasses import dataclass

In [56]:
@dataclass(frozen=True)
class DataValidationconfig:
    root_dir:Path
    status_file:str
    unzip_dir:Path
    all_schema:dict

In [57]:
from mlops.constants import *
from mlops.utils.common import read_yaml,create_directories

In [58]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH,
            parms_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.parms = read_yaml(parms_filepath)

        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> DataValidationconfig:
        
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationconfig(
            root_dir = config.root_dir,
            status_file = config.status_file,
            unzip_dir= config.unzip_dir,
            all_schema=schema
        )

        return data_validation_config
        


In [59]:
import os
from mlops import logger

In [64]:
class DataValidation:
    def __init__(self, config: DataValidationconfig):
        self.config = config
        self._ensure_directory_exists(os.path.dirname(self.config.status_file))

    def _ensure_directory_exists(self, directory: str):
        if not os.path.exists(directory):
            os.makedirs(directory)

    def validate_all_columns(self) -> bool:
        try:
            validation_status = True


            # Assuming self.config.unzip_dir is a file path, not a directory
            data = pd.read_csv(self.config.unzip_dir)
            schema = self.config.all_schema



            # Ensure all required columns are present
            for column in schema.keys():
                if column not in data.columns:
                    validation_status = False
                    self._write_status_file(False, f"Missing column: {column}")
                    return validation_status

            # Validate each column's data type
            for column, expected_dtype in schema.items():
                if column in data.columns:
                    actual_dtype = str(data[column].dtype)
                    if actual_dtype != expected_dtype:
                        validation_status = False
                        self._write_status_file(False, f"Column {column} is of type {actual_dtype}, expected {expected_dtype}")
                        return validation_status

            self._write_status_file(validation_status, "All columns validated successfully")
            return validation_status
        except Exception as e:
            self._write_status_file(False, f"Exception occurred: {str(e)}")
            raise e

    def _write_status_file(self, status: bool, message: str):
        with open(self.config.status_file, 'w') as f:
            f.write(f"Validation Status: {status}\n")
            f.write(message)

In [65]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e


[2024-07-19 18:44:37,137:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-07-19 18:44:37,144:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-07-19 18:44:37,146:INFO:common:yaml file: params.yaml loaded successfully]
[2024-07-19 18:44:37,147:INFO:common:created directory at: artifacts]
[2024-07-19 18:44:37,148:INFO:common:created directory at: artifacts/data_validation]
